# Scraping: récupération de la page d'accueil de la RTBF

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [La RTBF](https://www.rtbf.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/la-cgsp-annonce-une-greve-dans-les-services-publics-le-25-novembre-11616272',
  'La CGSP annonce une grève dans les services publics le 25\xa0novembre'],
 ['https://www.rtbf.be/article/nouvel-episode-de-la-saga-thiam-athletiek-vlaanderen-se-dit-pret-a-aller-au-tribunal-jusqu-au-bout-meme-11616379',
  'Nouvel épisode de la saga Thiam\xa0: Athletiek Vlaanderen se dit prêt à "aller au tribunal, jusqu’au bout même"'],
 ['https://www.rtbf.be/article/guerre-israel-gaza-la-paix-et-maintenant-la-suite-du-plan-trump-sous-d-epais-nuages-de-questions-11615936',
  'Guerre Israël-Gaza\xa0: la paix, et maintenant\xa0? La suite du plan Trump sous d’épais nuages de questions'],
 ['https://www.rtbf.be/article/the-voice-kids-saison-3-suivez-le-premier-blind-en-direct-11616127',
  'The Voice Kids saison 3 : suivez le premier blind en direct'],
 ['https://www.rtbf.be/article/raphael-collignon-apres-sa-victoire-sur-son-pote-zizou-bergs-je-vis-des-beaux-moments-depuis-deux-moi

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/la-cgsp-annonce-un...,La CGSP annonce une grève dans les services pu...
1,https://www.rtbf.be/article/nouvel-episode-de-...,Nouvel épisode de la saga Thiam : Athletiek Vl...
2,https://www.rtbf.be/article/guerre-israel-gaza...,"Guerre Israël-Gaza : la paix, et maintenant ? ..."
3,https://www.rtbf.be/article/the-voice-kids-sai...,The Voice Kids saison 3 : suivez le premier bl...
4,https://www.rtbf.be/article/raphael-collignon-...,Raphaël Collignon après sa victoire sur 'son p...
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,Les Niouzz
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Demain nous appartient
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Tarmac Comedy
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Ici tout commence


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

La CGSP annonce une grève dans les services publics le 25 novembre
Le syndicat socialiste des services publics, la CGSP,  annonce une nouvelle mobilisation  après celle de ce 14 octobre. Il s’agira d’une journée de grève pour la CGSP, précise Mme Reniers.
Les actions viseront à dénoncer les mesures gouvernementales qui concernent les fonctionnaires, notamment la réforme des pensions.
Les employés du service public devront faire plus de concessions, assure-t-elle. " Nous avons consacré beaucoup de temps aux négociations, mais nous n’avons pas constaté de concession concrète en réponse à nos demandes ", précise la syndicaliste.
La CGSP estime qu’il y a trop peu d’investissements dans les services publics . " Il s’agit de services qui nécessitent beaucoup de personnel, non seulement dans le secteur des soins de santé, mais aussi dans l’enseignement, les prisons, les pompiers et la police, entre autres ", explique Mme Reiners. Elle déplore des choix qui " consistent à réduire au minimum le

### Nettoyage du texte à l'aide d'expressions régulières

In [8]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Le syndicat socialiste des services publics, la CGSP,  annonce une nouvelle mobilisation  après celle de ce 14 octobre. Il s’agira d’une journée de grève pour la CGSP, précise Mme Reniers.
Les actions viseront à dénoncer les mesures gouvernementales qui concernent les fonctionnaires, notamment la réforme des pensions.
Les employés du service public devront faire plus de concessions, assure-t-elle. " Nous avons consacré beaucoup de temps aux négociations, mais nous n’avons pas constaté de concession concrète en réponse à nos demandes ", précise la syndicaliste.
La CGSP estime qu’il y a trop peu d’investissements dans les services publics . " Il s’agit de services qui nécessitent beaucoup de personnel, non seulement dans le secteur des soins de santé, mais aussi dans l’enseignement, les prisons, les pompiers et la police, entre autres ", explique Mme Reiners. Elle déplore des choix qui " consistent à réduire au minimum les services publics et le personnel ".
Le syndicat libéral SLFP doit

### Création d'un fichier avec le contenu de l'article


In [9]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp